In [ ]:
import json
import random
import torch
from pathlib import Path
from transformers import AutoModelForCausalLM, AutoTokenizer
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.live import Live

In [2]:
MODEL_ID = "HuggingFaceTB/SmolLM2-360M-Instruct"
DATA_PATH = Path("..") / "data" / "TOSDR" / "tosdr_markdowns_en.jsonl"
console = Console()

In [3]:
def load_smol():
    console.print(f"[bold yellow]Chargement de {MODEL_ID}...[/bold yellow]")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.bfloat16, # Très efficace sur GPU moderne
        device_map="auto"
    )
    return model, tokenizer

In [4]:
def get_smol_summary(model, tokenizer, text):
    # Format de prompt spécifique à SmolLM2 (Llama-style)
    messages = [
        {"role": "system", "content": "You are a legal expert. Classify clauses from the text as [GOOD], [NEUTRAL], [BAD], or [BLOCKER]."},
        {"role": "user", "content": f"Analyze this Terms of Service:\n\n{text[:2500]}"}
    ]
    
    # SmolLM2 utilise le template standard de Hugging Face
    input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    
    # Paramètres de génération pour un petit modèle (top_p plus bas pour éviter l'errance)
    outputs = model.generate(
        **inputs,
        max_new_tokens=512,
        temperature=0.2, 
        top_p=0.9,
        do_sample=True,
        repetition_penalty=1.1
    )
    
    return tokenizer.decode(outputs[0][len(inputs.input_ids[0]):], skip_special_tokens=True).strip()

In [5]:
model, tokenizer = load_smol()

# Chargement du dataset
with open(DATA_PATH, "r", encoding="utf-8") as f:
    samples = [json.loads(line) for line in f if "success" in line]

Chargement de HuggingFaceTB/SmolLM2-360M-Instruct...

`torch_dtype` is deprecated! Use `dtype` instead!


In [ ]:
for _ in range(3):
    item = random.choice(samples)
    console.print(f"\n[bold green]Test SmolLM2-360M - Service: {item['service_name']}[/bold green]")
    # print le markdown initial 
    console.print(Panel(item['markdown'], title=f'Markdown brut - {item["service_name"]}', border_style="green"))

    with console.status("[bold magenta]Génération en cours..."):
        result = get_smol_summary(model, tokenizer, item['markdown'])

    # Création d'un tableau pour un rendu "ToS;dr"
    table = Table(title=f"Rapport de clauses : {item['service_name']}", border_style="cyan")
    table.add_column("Analyse du modèle", style="white")

    # On sépare les lignes pour un affichage propre
    for line in result.split('\n'):
        if line.strip():
            table.add_row(line.strip())
            
    console.print(table)

Test SmolLM2-360M - Service: QwikAd.com

╭────────────────────────────────────────── Markdown brut - QwikAd.com ───────────────────────────────────────────╮
│ [Home](https://qwikad.com/)                                                                                     │
│                                                                                                                 │
│ [My Account](https://qwikad.com/acc_user_ads.php)                                                               │
│                                                                                                                 │
│ [Help](https://qwikad.com/help.php)                                                                             │
│  [Contact](https://qwikad.com/contact.php)                                                                      │
│                                                                                                                 │
│ 🔥                                                                                                              │
│                                                                                                                 │
│ LOWEST AD PRICES                                                                                                │
│ THIS MONTH ONLY!                                                                                                │
│                                                                                                                 │
│ [Post Free Ad](https://qwikad.com/post_local.php)  [Post Global Ad](https://qwikad.com/post_all.php)  [Post     │
│ Banner Ad](https://qwikad.com/post_banpost.php)                                                                 │
│                                                                                                                 │
│ SUPER SALE: Post One Paid Ad - Get One FREE! (through January 31st)  [Learn                                     │
│ More](https://qwikad.com/superpromo_page.php)                                                                   │
│                                                                                                                 │
│ ## QwikAd Classifieds                                                                                           │
│                                                                                                                 │
│ ## Sorry, this ad has been deleted or it doesn't exist                                                          │
│                                                                                                                 │
│ What would you like to do next?                                                                                 │
│                                                                                                                 │
│  [Return To Home Page](https://qwikad.com/)                                                                     │
│                                                                                                                 │
│  [Post A Local Ad](https://qwikad.com/post_local.php)                                                           │
│                                                                                                                 │
│  [Post A Global Ad](https://qwikad.com/post_all.php)                                                            │
│                                                                                                                 │
│  [Post A Banner Ad](https://qwikad.com/post_banpost.php)                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Rapport de clauses : 
     QwikAd.com      
┏━━━━━━━━━━━━━━━━━━━┓
┃ Analyse du modèle ┃
┡━━━━━━━━━━━━━━━━━━━┩
│ [GOOD]            │
└───────────────────┘

Test SmolLM2-360M - Service: CEUFast

╭──────────────────────────────────────────── Markdown brut - CEUFast ────────────────────────────────────────────╮
│ # CEUfast Terms of Service                                                                                      │
│                                                                                                                 │
│ Welcome to CEUfast.com. CEUfast provides continuing education for Nurses and other Healthcare Professional      │
│ throughout the United States and its Territories. By using the CEUfast.com web site ("Service"), or any         │
│ services of CEUfast, Inc. ("CEUfast"), you ("Customer") are agreeing to be bound by the following terms and     │
│ conditions ("Terms of Service"). If you are entering into this Agreement on behalf of a company or other Legal  │
│ Entity, you represent that you have the authority to bind such entity, its affiliates and all users who access  │
│ our services through your account to these Terms and Conditions.                                                │
│                                                                                                                 │
│ If CEUfast makes material changes to these Terms of Service, we will notify you by email or by posting a notice │
│ on our website before the changes are effective. Any new or improved features that augment or enhance the       │
│ current Service, including the release of new course material, tests, tools, and resources, shall be subject to │
│ the Terms of Service. Continued use of the Service after any such changes shall constitute your consent to such │
│ changes. You can review the most current version of the Terms of Service at any time at:  [CEUfast.com/TOS.]()  │
│                                                                                                                 │
│ ### Corporate Intelligence Gathering  *(notice to our competitors)*                                             │
│                                                                                                                 │
│ As used in these Terms of Service, the term "Competitor" means anyone who is employed or consults in the        │
│ business of designing, programming, marketing or selling continuing education and the term "Competitive         │
│ Purposes" means any and all activities related to the design, programming, marketing or sale of continuing      │
│ education, other than feedback given to CEUfast about the Service. Customer agrees not to discuss details of    │
│ the Service with, or provide screenshots, descriptions of the Service, or Service models to, anyone that may    │
│ use the information for Competitive Purposes.                                                                   │
│                                                                                                                 │
│ ### A. Account Terms                                                                                            │
│                                                                                                                 │
│ * You must be 13 years or older to use this Service.                                                            │
│ * You must be a human. Accounts registered by "bots" or other automated methods are not permitted.              │
│ * You must provide your name, mailing address, a valid email address, and any other information request in      │
│ order to complete the signup process.                                                                           │
│ * Your login may only be used by one person - a single login shared by multiple people is not permitted.        │
│ * You are responsible for maintaining the security of your account and password. CEUfast cannot and will not be │
│ liable for any loss or damage from your failure to comply with this security obligation.                        │
│ * One person or legal entity may not maintain more tha

Output()

In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "HuggingFaceTB/SmolLM2-360M-Instruct"

device = "cpu"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# for multiple GPUs install accelerate and do `model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")`
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

In [10]:
messages = [{"role": "user", "content": "What is gravity?"}]
input_text=tokenizer.apply_chat_template(messages, tokenize=False)
print(input_text)
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=50, temperature=0.2, top_p=0.9, do_sample=True)
print(tokenizer.decode(outputs[0]))


<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
What is gravity?<|im_end|>

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
What is gravity?<|im_end|>
<|im_start|>assistant
Gravity is a fundamental force of nature that causes objects with mass to attract each other. It is a universal force that acts between two objects with mass, regardless of their distance from each other. According to Einstein's theory of general


In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer
checkpoint = "HuggingFaceTB/SmolLM2-135M-Instruct"

device = "cpu"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# for multiple GPUs install accelerate and do `model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")`
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)


In [8]:
messages = [{"role": "user", "content": "What is gravity?"}]
input_text=tokenizer.apply_chat_template(messages, tokenize=False)
print(input_text)
inputs = tokenizer.encode(input_text, return_tensors="pt").to(device)
outputs = model.generate(inputs, max_new_tokens=50, temperature=0.2, top_p=0.9, do_sample=True)
print(tokenizer.decode(outputs[0]))


<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
What is gravity?<|im_end|>

<|im_start|>system
You are a helpful AI assistant named SmolLM, trained by Hugging Face<|im_end|>
<|im_start|>user
What is gravity?<|im_end|>
<|im_start|>assistant
Gravity is a fundamental force of nature that attracts objects with mass towards each other. It is a result of the interaction between the mass of an object and the distance between it and the center of the Earth. The force of gravity
